In [ ]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
import random

In [ ]:
orders = pd.read_csv('orders.csv')
couriers = pd.read_csv('partners_delays.csv')

In [ ]:
orders['date'] = pd.to_datetime(orders['date'])
couriers['dttm'] = pd.to_datetime(couriers['dttm'])

### What's going on here? We expect to receive a dataset with the date, delivery zones, number of couriers and number of orders. At the output, we get a dataset in which the columns look like data_area, where "data" is the day without hours (2021-04-01), and "area" is the delivery area. We also have a "time" column, which contains numbers from 8 to 23, these are delivery hours, it will say how many couriers we need.

In [ ]:
# I think the code could be cleaner and more efficient, but we didn't have time to think about it, 
# Eventually we didn't have time to predict the number of couriers
def optimize(dataset):
  index = 0
  
  # Here we want to find an index that will indicate the beginning and end of the day

  for area in dataset['delivery_area_id'].unique():
    for i in range(7):
      start_date = dataset['dttm'].loc[index]
      end_date = (dataset[(dataset['delivery_area_id'] == area) & (dataset['dttm'] < start_date + pd.DateOffset(days=1))]['dttm'].index)[-1]
      ord_count = dataset['orders'][index:end_date + 1].sum()
      dates = {}
      for i in range(index, end_date+1):
        dates[str(new.loc[i]['dttm'])] = new.loc[i]['orders']

      max_workers = int(new['partners_cnt'][index:end_date+1].max()) # Max count of available couriers per day

      worker = {}
      for i in range(max_workers):
        worker['{}'.format(i)] = 1 # Creating list of couriers per day
      workers, pay = gp.multidict(worker)

      c = 0
      availability = gp.tuplelist([])  # Availability of couriers(when)
      for i in range(index, end_date+1):
        for k in range(int(new.loc[i]['partners_cnt'])):
          if c == max_workers:
            c = 0
          availability.append(('{}'.format(workers[c]),'{}'.format(new.loc[i]['dttm'])))
          c+= 1

      index = end_date + 1
      shifts, shiftRequirements = gp.multidict(dates) # Creating shifts

      m = gp.Model("workforce5")
      x = m.addVars(availability, vtype=GRB.BINARY, name="x")
      slacks = m.addVars(shifts, name="Slack")
      totSlack = m.addVar(name='totSlack')
      totShifts = m.addVars(workers, name="TotShifts")
      shift_reqmts = m.addConstrs((x.sum('*',s) + slacks[s] == shiftRequirements[s] for s in shifts), name='shiftRequirement')
      num_temps = m.addConstr(totSlack == slacks.sum(), name='totSlack')
      num_shifts = m.addConstrs((totShifts[w] == x.sum(w,'*') for w in workers), name='totShifts')
      minShift = m.addVar(4, name='minShift')
      maxShift = m.addVar(8,name='maxShift')
      min_constr = m.addGenConstrMin(minShift, totShifts, name='minShift')

      """
      If we remove '#' from next line of code we get a limit on max shift duration 8 hours
          But there are days when this condition cannot be met, there are significantly fewer couriers than orders, and either someone
          Does not receive his order, or someone works more than 8 hours (That's how I understood it)
      """

      #max_constr = m.addGenConstrMin(maxShift, totShifts, name='maxShift')
      m.ModelSense = GRB.MINIMIZE
      m.setObjectiveN(totSlack, index=0, priority=2, reltol=0.2, name='TotalSlack')
      m.setObjectiveN(maxShift - minShift, index=1, priority=1, name='Fairness')
      m.write('workforce.lp')
      m.optimize()
      # The Status attribute  provides current optimization status of the model object m
      # In workforce model, we check if the model is infeasible or unbounded and report this situation
      status = m.Status
      if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE  or status == GRB.Status.UNBOUNDED:
          print('The model cannot be solved because it is infeasible or unbounded')
          sys.exit(0)
      # If the optimization status of the model is not optimal for some other reason, we report that 
      # situation.
      if status != GRB.Status.OPTIMAL:
          print('Optimization was stopped with status ' + str(status))
          sys.exit(0)

      # Print total slack and the number of shifts worked for each worker
      # The KPIs for this optimization number is the number of extra worked required to satisfy 
      # demand and the number of shifts that each employed worker is working.
      solution = {}
      shifts_sol = {}
      solution['Total slack required'] = str(totSlack.X)
      assignments_all = {}
      gant={}

      assignments = dict()
      for [w, s] in availability:
          if x[w, s].x == 1:
              if w in assignments:
                  assignments[w].append(s)
              else:
                  assignments[w] = [s]

      
      # Remove '#' if you want to see distribution of shifts of couriers on plots >>>
      
      #print(pd.DataFrame.from_records(list(solution.items()), columns=['KPI', 'Value']))
      #print('-'*50)

      for w in workers:
          shifts_sol[w]=totShifts[w].X
          assignments_all[w]=assignments.get(w, [])

      #print('Shifts')
      #print(pd.DataFrame.from_records(list(shifts_sol.items()), columns=['Worker', 'Number of shifts']))
          
      y_pos = np.arange(len(shifts_sol.keys())) 
      #plt.bar(y_pos,shifts_sol.values() , align='center')
      #plt.xticks(y_pos, shifts_sol.keys())
      #plt.show()

      #print('-'*50)
      for w in assignments_all:
          gant[w] = [w]
          for d in shifts:
              gant[w].append('*' if d in assignments_all[w] else '-')

      #print('Assigments')
      #print('Symbols: \'-\': not working, \'*\': working')
      #pd.set_option('display.width', 1000)
      #print(pd.DataFrame.from_records(list(gant.values()), columns=['worker']+shifts))
      #print(pd.DataFrame.from_records(list(gant.values())))
      df['{}'.format(str(start_date)[:10])+"_"+str(area)] = 0
      for i in range(len(shifts)):
        cour = 0
        for k in workers:
          if gant[k][i +1 ] == '*':
            cour += 1

        ind = df[(df['time'] == pd.to_datetime(shifts[i]).hour)].index
        df['{}'.format(str(shifts[i])[:10])+"_"+str(area)][ind] = cour
  return df

In [ ]:
optimize(couriers)